In [21]:
import ipywidgets as widgets
from IPython.display import display, HTML
from fractions import Fraction

# NOTES - Lengths
note_durations = [
    ("𝅝",     Fraction(1)),        # Whole
    ("𝅗𝅥•",    Fraction(3, 4)),     # Dotted Half
    ("𝅗𝅥",     Fraction(1, 2)),     # Half
    ("𝅘𝅥•",    Fraction(3, 8)),     # Dotted Quarter
    ("𝅗𝅥³",    Fraction(1, 3)),     # Triplet Half
    ("𝅘𝅥",     Fraction(1, 4)),     # Quarter
    ("𝅘𝅥𝅮•",    Fraction(3, 16)),    # Dotted 8th
    ("𝅘𝅥³",    Fraction(1, 6)),     # Triplet Quarter
    ("𝅘𝅥𝅮",     Fraction(1, 8)),     # 8th
    ("𝅘𝅥𝅯•",    Fraction(3, 32)),    # Dotted 16th
    ("𝅘𝅥𝅮³",    Fraction(1, 12)),    # Triplet 8th
    ("𝅘𝅥𝅯",     Fraction(1, 16)),    # 16th
    #("𝅘𝅥𝅰•",    Fraction(3, 64)),    # Dotted 32nd
    ("𝅘𝅥𝅯³",    Fraction(1, 24)),    # Triplet 16th
    ("𝅘𝅥𝅰",     Fraction(1, 32)),    # 32nd
    #("1⁄64",  Fraction(1, 64)),    # 64th
    #("𝅘𝅥𝅰³",    Fraction(1, 48)),    # Triplet 32nd
    #("1⁄64•", Fraction(3, 128)),   # Dotted 64th
]

# DIVISIONS - Maximum Subdivisions
division_steps = [
    ("𝅘𝅥𝅮 8th", 8),
    ("Triplet 8th", 12),
    ("𝅘𝅥𝅯 16th", 16),
    ("Triplet 16th", 24),
    ("𝅘𝅥𝅰 32nd", 32),
    ("Polymetric", 48),
]

# LAYOUT FORMATTING - SLIDERS **slider_format_etc
slider_format_hor = {
    'orientation': 'horizontal',
    'layout': widgets.Layout(width='400px'),
    'style': {'description_width': '120px'}
}
slider_format_ver = {
    'orientation': 'vertical',
    'layout': widgets.Layout(width='40px'),
    'style': {'description_width': '120px'}
}
slider_format_opt = {
    'orientation': 'horizontal',
    'layout': widgets.Layout(width='400px'),
    'style': {'description_width': '120px'}
}

# GLOBAL SLIDERS
tempo_slider = widgets.IntSlider(description='Tempo', min=30, max=300, step=1, value=90, **slider_format_hor)
signum_slider = widgets.IntSlider(description='Count', min=2, max=15, step=1, value=4, **slider_format_hor)
signote_slider = widgets.SelectionSlider(description='Beat Unit', options=[4, 8, 16], value=4, **slider_format_hor)
division_slider = widgets.SelectionSlider(description='Subdivision', options=division_steps, value=48, **slider_format_hor) # 'Max Division', options=[8, 16, 32, 12, 24, 48], value=8)
measures_slider = widgets.IntSlider(description='Measures', min=1, max=12, step=1, value=1, **slider_format_hor)

# TRACK SPECIFIC SLIDERS - Global Helper
def make_param_sliders(name='', note_val=Fraction(1,4), offset_val=0.0, group_val=0, pattern_val=0):
    return [
        widgets.SelectionSlider(description=f'Note', options=note_durations, value=note_val, **slider_format_ver),
        widgets.FloatSlider(description=f'Offset', min=0.0, max=1.0, step=0.01, value=offset_val, readout_format='.0%', **slider_format_ver),
        widgets.IntSlider(description=f'Group', min=0, max=100, step=1, value=group_val, **slider_format_ver), # NEXT TO DO
        widgets.IntSlider(description=f'Pattern', min=0, max=100, step=1, value=pattern_val, **slider_format_ver), # NEXT TO DO
    ]

# GROUP SLIDERS - Layout
h_params = make_param_sliders("Hi Hat-", note_val=Fraction(1, 8), offset_val=0, group_val=1, pattern_val=0)
s_params = make_param_sliders("Snare-", note_val=Fraction(1, 2), offset_val=0.5, group_val=1, pattern_val=0)
k_params = make_param_sliders("Kick-", note_val=Fraction(1, 2), offset_val=0.0, group_val=1, pattern_val=0)

all_sliders = [tempo_slider, signum_slider, signote_slider, division_slider, measures_slider,
               *h_params,*s_params,*k_params
]

sliders_form = [
    tempo_slider, 
    signum_slider, 
    signote_slider, 
    division_slider, 
    measures_slider
]
sliders_hihat = [*h_params]
sliders_snare = [*s_params]
sliders_kick = [*k_params]

box1 = widgets.VBox([widgets.Label("Main Form"), widgets.VBox(sliders_form)])
box2 = widgets.VBox([widgets.Label("Hi Hat"), widgets.HBox(sliders_hihat)])
box3 = widgets.VBox([widgets.Label("Snare"), widgets.HBox(sliders_snare)])
box4 = widgets.VBox([widgets.Label("Kick"), widgets.HBox(sliders_kick)])

output = widgets.Output()

# PATTERN GENERATOR - TODO; add groupings and patterns, stickings, etc
def generate_measure(signum, signote, division, tempo=None, measures=1, p1=None, p2=None, p3=None, p4=None):
        # Generate blank string of ZEROs
    binary_length = int((signum * division) / signote)
    pattern = ['0'] * binary_length

        # NOTES INJECTOR - Insert ONE 
    if p1 and p1 > 0:
        steps_per_hit = int(division * p1)
        if steps_per_hit > 0:
            for i in range(0, binary_length, steps_per_hit):
                pattern[i] = '1'

        # OOFFSET - NOTE: Make the offset based off of a note value + quantity
    offset_wdiv = int(division * p2 * p1) # if p2 = 50% and DIVISION = 8, 16, 32 and p1 = 1/4 then offset = 1, 2, 4
    if offset_wdiv > 0:
        pattern = (['0'] * offset_wdiv) + pattern[:-offset_wdiv] # Relate this to Max Subdiv
        if offset_wdiv > binary_length:
            pattern = pattern[:-(offset_wdiv - binary_length)]

        # MEASURES x
    pattern = pattern * measures
    return pattern
        
        # GROUPING
    # TODO; At every '1' insert '#' of 1s ahead or behind that initial '1'. Maybe just using the Max Division as steps, or have it driven by a Note Length
    
        # PATTERN
    # TODO; Fuck this one is hard. 

######### Idea for Count and/or Sticking
#s = "1000100010001000"
#counter = 1
#result = ""
#
#for char in s:
#    if char == '1':
#        result += str(counter)
#        counter += 1
#    else:
#        result += char
#
#print(result)

# UPDATER, FORMAT, DISPLAY
def on_slider_change(change=None):
    with output:
        output.clear_output()

        # GET VALUES
        signum = signum_slider.value
        signote = signote_slider.value
        division = division_slider.value
        tempo = tempo_slider.value
        measures = measures_slider.value

        # GET PARAMETERS
        def get_params(param_group):
            return [w.value for w in param_group]

        p_h = get_params(h_params)
        p_s = get_params(s_params)
        p_k = get_params(k_params)

        # FORMAT PATTERNS
        def to_formatd(binary, char='o'):
            return ''.join(binary).replace('1', char).replace('0', '-')

        h_bin = generate_measure(signum, signote, division, tempo, measures, *p_h)
        s_bin = generate_measure(signum, signote, division, tempo, measures, *p_s)
        k_bin = generate_measure(signum, signote, division, tempo, measures, *p_k)

        h_conv = to_formatd(h_bin, 'x')  # x for hi-hat normal  (options: x, o, X, c, r, b, m, s, n, N)
        s_conv = to_formatd(s_bin, 'o')  # o for snare normal   (options: o, O, g, x, b, f)
        k_conv = to_formatd(k_bin, 'o')  # o for kick normal    (options: o, x, X)

        # DISPLAY
        # GENERATE LINK to GrooveScribe
        groove_url = (
            f"https://www.mikeslessons.com/groove/?"
            f"TimeSig={signum}/{signote}&"
            f"Div={division}&"
            f"Tempo={tempo}&"
            f"Measures={measures}&"
            f"H=|{h_conv}|&"
            f"S=|{s_conv}|&"
            f"K=|{k_conv}|" # TO DO; add "&Stickings=|RLBccc------|" Right, Left, Both, Count
        )
        display(HTML(f'<a href="{groove_url}" target="_blank">► LINK TO PLAY/EDIT (via:GrooveScribe)</a>'))

        # ASCII - A readable ASCII art version of the notation. Nice to add note counts dirived from the Steps Per Hit against Note Value (ie, 1e&a2e&a)
        #print(f"")

        # READABLE
        print(f"\nH:    |{h_conv}|")
        if signum < 10 and signote < 10:
            print(f"S: {signum}/{signote}|{s_conv}|")
        elif signum < 10 or signote < 10:
            print(f"S:{signum}/{signote}|{s_conv}|")
        else:
            print(f"S{signum}/{signote}|{s_conv}|")
        print(f"K:    |{k_conv}|")

        # BINARY
        print(f"\nBin H: {''.join(h_bin)}")
        print(f"Bin S: {''.join(s_bin)}")
        print(f"Bin K: {''.join(k_bin)}")

        # HEX
        try:
            hex_h = f"{int(''.join(h_bin), 2):x}".upper()
            hex_s = f"{int(''.join(s_bin), 2):x}".upper()
            hex_k = f"{int(''.join(k_bin), 2):x}".upper()
            print(f"\nHex H: {hex_h}")
            print(f"Hex S: {hex_s}")
            print(f"Hex K: {hex_k}")
        except:
            print("Error converting to hex.")

# OBSERVE!
for s in all_sliders:
    s.observe(on_slider_change, names='value')

# OPTIONAL FORMATTING -  Box styles
def style_box(box):
    return widgets.VBox([box], layout=widgets.Layout(border='3px solid lightgray', padding='5px', align_items='center'))
styled_box1 = style_box(box1)
styled_box2 = style_box(box2)
styled_box3 = style_box(box3)
styled_box4 = style_box(box4)

# UI - Layout
display(widgets.HBox([styled_box1, styled_box2, styled_box3, styled_box4]))
#display(widgets.HBox([box1, box2, box3, box4])) # NO STYLE
display(output)

# Trigger on load
on_slider_change()


Output()